# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_data_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_data_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_data_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Table is modeled based on a query asking the following information:
##   Give me the artist, song title and song's length in the music app history
##   that was heard during sessionId = 338, and itemInSession = 4
##
## Primary key is composed of a session id and the item within that session.
## This combination of columns creates a unique row and composes the columns
## that are utilized in the WHERE clause.
query = "CREATE TABLE IF NOT EXISTS song_info_by_session_item "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_data_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session_item (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT artist_name, song_title, song_length FROM song_info_by_session_item WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows))
print(df)

  artist_name                       song_title  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Table is modeled based on a query asking the following information:
##   Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
##   for userid = 10, sessionid = 182
##
## Primary key is composed of the user and session ids as the partition key and the item in the session
## as an addded clustering column.  The item in session is needed to sort the results by the required
## sort order requested of the query.  The combination of the three columns creates a unique row. The user
## id and session id need to be part of the primary key as they are part of the WHERE clause.
query = "CREATE TABLE IF NOT EXISTS song_info_by_user_session "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, \
                PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
               

In [12]:
file = 'event_data_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_user_session (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
query = "SELECT artist_name, song_title, first_name, last_name FROM song_info_by_user_session WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows))
print(df)

         artist_name                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


In [14]:
## Table is modeled based on a query asking the following information: 
##   Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
##
## The primary key is composed of the song title, session id and user id.  This combination of column
## produces a unique row.  The inclusion of the song_title is required as it is part of the WHERE clause
## The addtion of item in session is required as a combination of song title and user and session id's may 
## not be unique as a user id may listen to the same song in the same session.  It also prevents the case where
## a song title is not unique.
query = "CREATE TABLE IF NOT EXISTS users_by_song "
query = query + "(song_title text, session_id int, user_id int, item_in_session int, first_name text, last_name text, \
                PRIMARY KEY (song_title, session_id, user_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                             

In [15]:
file = 'event_data_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song_title, session_id, user_id, item_in_session, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[8]), int(line[10]), int(line[3]), line[1], line[4]))

In [16]:
query = "SELECT first_name, last_name FROM users_by_song WHERE song_title='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows))
print(df)

   first_name last_name
0        Sara   Johnson
1  Jacqueline     Lynch
2       Tegan    Levine


### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE song_info_by_session_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE song_info_by_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE users_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()